In [6]:
import sys
import numpy as np

# Obtener valor de línea de comandos
if len(sys.argv) > 1:
    value = int(sys.argv[1])
else:
    value = 5*10**7

print(f"Executing with value = {value}\n")

ValueError: invalid literal for int() with base 10: '-f'

## Reduction: the sum of the elements of an array

In [7]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**7

X = np.random.rand(value)

# Para imprimir los primeros valores del array

#print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


Time taken by reduction operation using a function: 5.56 s ± 211 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001033.95208851

Time taken by reduction operation using numpy.sum(): 18.9 ms ± 26.8 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001033.952088803 
 


## Operación Multiprocessing con Pool

In [8]:
from multiprocessing import Pool

def reduc_operation_chunk(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

def parallel_reduction(A, num_processes):
    """Parallel reduction using multiprocessing.Pool"""
    chunk_size = A.size // num_processes
    chunks = [A[i*chunk_size:(i+1)*chunk_size] for i in range(num_processes)]
    
    if A.size % num_processes != 0:
        chunks[-1] = A[(num_processes-1)*chunk_size:]
    
    with Pool(num_processes) as pool:
        results = pool.map(reduc_operation_chunk, chunks)
    
    return sum(results)

# Con 2 procesos
print("Multiprocessing with 2 processes:")
tiempo = %timeit -r 2 -o -q parallel_reduction(X, 2)
print("Time taken:", tiempo)
print(f"Result: {parallel_reduction(X, 2)}\n")

# Con 4 procesos
print("Multiprocessing with 4 processes:")
tiempo = %timeit -r 2 -o -q parallel_reduction(X, 4)
print("Time taken:", tiempo)
print(f"Result: {parallel_reduction(X, 4)}\n")

Multiprocessing with 2 processes:
Time taken: 2.75 s ± 20.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result: 25001033.95209656

Multiprocessing with 4 processes:
Time taken: 1.57 s ± 10.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result: 25001033.952088863



## Numba secuencial y paralelo

In [9]:
from numba import njit

@njit
def reduc_operation_numba(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Primera ejecución para compilar
_ = reduc_operation_numba(X)

# Medir tiempo
print("Numba with @njit (sequential):")
tiempo = %timeit -r 2 -o -q reduc_operation_numba(X)
print("Time taken:", tiempo)
print(f"Result: {reduc_operation_numba(X)}\n")

Numba with @njit (sequential):
Time taken: 49.9 ms ± 102 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Result: 25001033.95208851



In [10]:
from numba import prange
import os

@njit(parallel=True)
def reduc_operation_numba_parallel(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

# Primera ejecución para compilar
os.environ['OMP_NUM_THREADS'] = '4'
_ = reduc_operation_numba_parallel(X)

# Con 2 núcleos
os.environ['OMP_NUM_THREADS'] = '2'
print("Numba with @njit(parallel=True) and prange - 2 cores:")
tiempo = %timeit -r 2 -o -q reduc_operation_numba_parallel(X)
print("Time taken:", tiempo)
print(f"Result: {reduc_operation_numba_parallel(X)}\n")

# Con 4 núcleos
os.environ['OMP_NUM_THREADS'] = '4'
print("Numba with @njit(parallel=True) and prange - 4 cores:")
tiempo = %timeit -r 2 -o -q reduc_operation_numba_parallel(X)
print("Time taken:", tiempo)
print(f"Result: {reduc_operation_numba_parallel(X)}\n")

Numba with @njit(parallel=True) and prange - 2 cores:
Time taken: 11.6 ms ± 34.1 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Result: 25001033.952088855

Numba with @njit(parallel=True) and prange - 4 cores:
Time taken: 11.6 ms ± 16.5 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Result: 25001033.952088855



## Resultados de los analisis

Al comparar los resultados obtenidos para tamaños de entrada de 10^8 y 10^9 elementos, se observa como la velocidad del código original en Python apenas varía. En ambos casos, el tiempo de ejecución se mantiene alrededor de los 8.5–8.7 segundos, independientemente del número de núcleos asignados. Por ello, el la mayoría del coste no está relacionado con el tamaño del problema, sino con la propia ejecución secuencial en Python, que no es capaz de aprovechar ni el paralelismo ni el incremento de recursos computacionales.

En el caso de la operación vectorizada con numpy.sum(), los tiempos de ejecución son prácticamente idénticos para 10^8 y 10^9 elementos, situándose en torno a los 32 milisegundos. Esta ausencia de diferencias significativas demuestra la alta eficiencia de las rutinas internas de NumPy. El aumento del tamaño del array en un orden de magnitud no penaliza el rendimiento, ya que la operación está limitada principalmente por la transferencia de datos en memoria y no por la carga computacional.

Al analizar la versión paralelizada mediante multiprocessing, se observa que el incremento de tamaño de 10^8 a 10^9 no introduce un aumento proporcional del tiempo de ejecución. Con 2 procesos, los tiempos se mantienen alrededor de los 5.5–6 segundos, mientras que con 4 procesos se reducen a aproximadamente 3–3.4 segundos en ambos tamaños. Esto indica que el coste dominante en multiprocessing sigue siendo encontrandose asociado a la creación de procesos y a la comunicación entre ellos, de modo que el aumento del número de elementos no modifica sustancialmente el comportamiento global del algoritmo.

La versión secuencial optimizada con Numba (@njit) muestra un comportamiento especialmente estable al pasar de 10^8 a 10^9 elementos. En ambos casos, los tiempos de ejecución se sitúan en el rango de 55–70 milisegundos, lo que evidencia que la compilación JIT elimina por completo el overhead de Python y genera código máquina altamente eficiente. El aumento del tamaño del problema no supone una desventaja, ya que el cálculo es lo suficientemente simple como para estar limitado principalmente por el acceso a memoria.

Finalmente, la versión paralela con Numba y prange presenta el mejor comportamiento en términos de escalabilidad. Al comparar 10^8 y 10^9 elementos, los tiempos de ejecución se mantienen en el orden de 15–20 milisegundos, especialmente al utilizar 4 núcleos. Esto indica que la combinación de compilación JIT y paralelización automática permite absorber el incremento del tamaño del problema sin degradar el rendimiento. Además, para 10^9 elementos, el paralelismo se aprovecha de forma más efectiva, ya que existe suficiente carga de trabajo para amortizar completamente el coste del paralelismo.

Por ello, las soluciones basadas en NumPy y Numba con prange, muestran como el uso de código compilado y paralelizado es importante para mantener un rendimiento elevado en operaciones de reducción sobre grandes volúmenes de datos.